# Monte Carlo Methods
Probability involves the generation of random variables from known distributions. Statistics deals with estimation of distribution parameters from actual data. With estimated distributions in hand, we proceed to the next step, which is the simulation of random variables for the purpose of risk management.

**Monte Carlo (MC)** simulations are central to financial engineering and risk management. They allow financial engineers to price complex financial instruments and risk managers to build the distribution of portfolios that are too complex to model analytically.

Simulation results depend heavily on the model's assumptions: the shape of the distribution, the parameters, and the pricing functions.

- [Simulations with One Random Variable](#simulation_with_one_random_variable)
- [Implement Simulations](#implement_simulations)

## <a name="simulation_with_one_random_variable">Simulation with One Random Variable</a>
Simulations involve creating artificial random variables with properties similar to those of the risk factors in the portfolio. These include stock prices, exchange rates, bond yields or prices, and commodity prices.
### Simulate Markov Processes
In efficient markets, financial prices should display a random walk pattern. More precisely, prices are assumed to follow a **Markov process**, which is a particular stochastic process independent of its past history; the entire distribution of the future price relies on the current price only. The past history is irrelevant. These processes are built from the following components, described in order of increasing complexity:
- **The Wiener process:** this describes a variable $\Delta z$, whose change is measured over the interval $\Delta t$ such that its mean change is $0$ and variance proportional to $\Delta t$:

$\Delta z \sim N(0, \Delta t)$

If $\epsilon$ is a standard normal variable $N(0, 1)$, this can be written as $\Delta z = \epsilon \sqrt{\Delta t}$. In addition, the increment $\Delta z$ are independent across time.
- **The generalized Wiener process:** this describes a variable $\Delta x$ built up from a Wiener process, with, in addition, a constant trend $a$ per unit of time and volatility $b$:

$\Delta x = a \Delta t + b \Delta z$

A particular case is the **martingale**, which is a zero-drift stochastic process, $a = 0$, which leads to $E(\Delta x) = 0$. This has the convenient property that the expectation of a future value is the current value

$E(x_{T}) = x_{0}$

- **The Ito process:** this describes a generalized Wienee process, whose trend and volatility depend on the current value of the underlying variable and time:

$\Delta x = a(x, t)\Delta t + b(x, t)\Delta z$

This is a Markov process because the distribution depends only on the current value of the random variable $x$, as well as time. In addition, the innovation in this process has a normal distribution.

### The geometric Brownian motion (GBM)
A particular example of Ito process is the **geometric Brownian motion (GBM)**, which is described for the variable $S$ as 

$\Delta S = \mu S \Delta t + \sigma S \Delta z$

The process is geometric because the trend and volatility terms are proportional to the current value of $S$. This is typically the case for stock prices, for which rates of return appear to be more stationary than raw dollar returns, $\Delta S$. It is also used for currencies. Because $\Delta S/S$ represents the capital appreciation only, abstracting from dividend payments, $\mu$ represents the expected total rate of return on the asset minus the rate of income payment, or dividend yield in the case of stocks.

This model is particularly important because it is the underlying process for the Black-Scholes formula. The key feature of this distribution is the fact that the volatility is proportional to $S$. This ensures that the stock price will stay positive. Indeed, as the stock price falls, its variance decreases, which makes it unlikely to experience a large down move that would push the price into negative values. As the limit if this model is a normal distribution for $dS/S = dln(S)$, $S$ follows a **lognormal distribution**.

This process implies that, over an interval $T - t = \tau$, the logarithm of the ending price is distributed as

$ln(S_{T}) = ln(S_{t}) + (\mu - \sigma^{2}/2)\tau + \sigma\sqrt{\tau}\epsilon$

where $\epsilon$ is a standardized normal variable.

Whether a lognormal distribution is much better than the normal distribution depends on the horizon considered. If the horizon is one day only, the choice of the lognormal versus normal assumption does not really matter. It is highly unlikely that the stock price would drop below $0$ in one day, given typical volatilities. However, if the horizon is measured in years, the two assumptions do lead to different results. The lognormal distribution is more realistic as it prevents prices from turning negative.

In simulations, this process is approximated by small steps with a normal distribution with mean and variance given by

$\frac{\Delta S}{S} \sim N(\mu\Delta t, \sigma^{2}\Delta t)$

To simulate the future price path for $S$, we start from the current price $S_{t}$ and generate a sequence of independent standard normal variables $\epsilon$, for $i = 1, 2,..., n$. The next price, $S_{t+1}$, is built as $S_{t+1} = S_{t} + S_{t}(\mu\Delta t + \sigma\epsilon_{1}\sqrt{\Delta t})$. The following price, $S_{t+2}$, is taken as $S_{t+1} + S_{t+1}(\mu\Delta t+\sigma\epsilon_{2}\sqrt{\Delta t})$, and so on until we reach the target horizon, at which point the price $S_{t+n} = S_{T}$ should have a distribution close to the lognormal.

Define $K$ as the number of **replications**, or random trials. Each leads to a simulated final value $S^{k}_{T}$. This generates a distribution of simulated prices $S_{T}$. With just one step $n = 1$, the distribution must be normal. As the number of steps $n$ grows large, the distribution tends to a lognormal distribution.

While very useful for modeling stock prices, this model has shortcomings. Price increments are assumed to have a normal distribution. In practice, we observe that price changes for most financial assets typically have fatter tails than the normal distribution. Returns may also experience changing variances.

In addition, as the time interval $\Delta t$ shrinks, the volatility shrinks as well. This implies that large discontinuities cannot occur over short intervals. In reality, some assets experience discrete jumps, such as commodities, or securities issued by firms that go bankrupt. In such cases, the stochastic process should be changed to accommodate these observations.

### Draw random variables
Most spreadsheets or statistical packages have functions that can generate uniform or standard normal random variables. This can be easily extended to distributions that better reflect the data (e.g., with fatter tails or nonzero skewness).

The methodology involves the inverse cumulative probability distribution function ($p.d.f.$). Take the normal distribution as an example. By definition, the cumulative $p.d.f.$ $N(x)$ is always between $0$ and $1$. Because we have an analytical formula for this function, it can be easily inverted.

First, we generate a uniform random variable $u$ drawn from $U(0, 1)$. Next, we compute $x$ such that $u = N(x)$, or $x = N^{-1}(u)$. The variable can be transformed into any normal variable by multiplying by the standard deviation and adding the mean. More generally, any distribution function can be generated as long as the cumulative distribution function can be inverted.
### Simulate yields
The $GBM$ process is widely used for stock prices and currencies. Fixed-income products are another matter, however.

Bond prices display long-term reversion to the face value, which represents the repayment of principal at maturity (assuming there is no default). Such a process is inconsistent with $GBM$ process, which displays no such mean reversion. The volatility of bond prices also changes in a predictable fashion, as duration shrinks to $0$. Similarly, commodities often display mean reversion.

These features can be taken into account by modeling bond yields directly in a first step. In the next step, bond prices are constructed from the value of yields and a pricing function. The dynamics of interest rates $r_{t}$ can be modeled by 

$\Delta{r_{t}} = \kappa(\theta - r_{t})\Delta t + \sigma r_{t}^{\gamma}\Delta z_{t}$

where $\Delta Z_{t}$ is the usual Wiener process. Here, we assume that $0 \leq \kappa < 1, \theta \geq 0, \sigma \geq 0$. Because there is only one stochastic variable for yields, the model is called a **one-factor model**.

This Markov process has a number of interesting features. First, it displays mean reversion to a long-run value of $\theta$. The parameter $\kappa$ governs the speed of mean reversion. When the current interest rate is high (i.e., $r_{t} > \theta$), the model creates a negative drift $\kappa(\theta - r_{t})$ toward $\theta$. Conversely, low current rates create a positive drift toward $\theta$.

The second feature is the volatility process. This model includes the **Vasicek model** when $\gamma = 0$. Changes in yields are normally distributed because $\Delta r$ is then a linear function of $\Delta z$, which is itself normal. The Vasicek model is particular convenient because it leads to closed-form solutions for many fixed-income products. The problem, however, is that it could potentially lead to negative interest rates when the initial rate starts from a low value. This is because the volatility of the change in rates does not depend on the level, unlike that in the geometric Brownian motion.

The $\Delta r_{t}$ equation is more general, however, because it includes a power of the yield in the variance function. With $\gamma = 1$, this is the **lognormal model**. Ignoring the trend, this gives $\Delta r_{t} = \sigma r_{t}\Delta z_{t}$, or $\Delta r_{t}/r_{t} = \sigma \Delta z_{t}$. This implies that the rate of change in the yield $dr/r$ has a fixed variance. Thus, as with the $GBM$ model, smaller yields lead to smaller movements, which makes it unlikely the yield will drop below $0$. This model is more appropriate than the normal model when the initial yield is close to $0$.

With $gamma = 0.5$, this is the **Cox, Ingersoll, and Ross (CIR) model**. Ultimately, the choice of the exponent $\gamma$ is an empirical issue. Recent research has shown that $\gamma = 0.5$ provides a good fit to the data.

This class of model is known as **equilibrium models**. They start with some assumptions about economic variables and imply a process for the short-term interest rate $\gamma$. These models generate a predicted term structure, whose shape depends on the model parameters and the initial short rate. The problem with these model, however, is that they are not flexible enough to provide a good fit to today's term structure. This can be viewed as unsatisfactory, especially by practitioners who argue they cannot rely on a model that cannot be trusted to price today's bonds.

In contrast, **no-arbitrage models** are designed to be consistent with today's term structure. In this class of models, the term structure is an input into the parameter estimation. The earliest model of this type was the **Ho and Lee model**:

$r_{t} = \theta(t)\Delta t + \sigma \Delta z_{t}$

where $theta(t)$ is a function of time chosen so that the model fits the initial term structure. This was extended to incorporate mean reversion in the **Hull and White model**:

$\Delta r_{t} = [\theta(t) - ar_{t}]\Delta t + \sigma \Delta z_{t}$

Finally, the **Heath, Jarrow, and Morton model** goes one step further and assumes that the volatility is a function of time.

The downside of these no-arbitrage models is that they do not impose any consistency between parameters estimated over different dates. The function $\theta(t)$ could be totally different from one day to the next, which is illogical. No-arbitrage models are also more sensitive to outliers, or data errors in bond prices used to fit the term structure.

### Binomial trees
Simulations are very useful to mimic the uncertainty in risk factors, especially with numerous risk factors. In some situations, however, it is also useful to describe the uncertainty in prices with discrete trees. When the price can take one of two steps, the tree is said to be **binomial**.

The binomial model can be viewed as a discrete equivalent to the geometric Brownian motion. As before, we subdivide the horizon $T$ into $n$ intervals $\Delta t = T/n$. At each node, the price is assumed to go either up with probability $p$ or down with probability $1 - p$.

The parameters $u, d, p$ are chosen so that, for a small time interval, the expected return and variance equal those of the continuous process. One could choose 

$u = e^{\sigma\sqrt{\Delta t}}, d = (1/u), p = \frac{e^{\mu \Delta t - d}}{u - d}$

This matches the mean, for example,

$E[\frac{S_{1}}{S_{0}}] = pu + (1 - p)d = \frac{e^{\mu\Delta t} - d}{u - d}u + \frac{u - e^{\mu\Delta t}}{u - d}d = \frac{e^{\mu\Delta t}(u - d) - du + ud}{u - d} = e^{\mu \Delta t}$

## <a name="implement_simulations">Implement simulations</a>
### Simulation for VAR
Implementing Monte Carlo (MC) methods for risk management follows these steps:
1. Choose a stochastic process for the risk factor price S (i.e., its distribution and parameters, starting from the current value $S_{t}$).
2. Generate pseudo-random variables representing the risk factor at the target horizon, $S_{T}$.
3. Calculate the value of the portfolio at the horizon, $F_{T}(S_{T})$.
4. Repeat steps 2 and 3 as many times as necessary. Call $K$ the number of replications.

These steps create a distribution of values, $F^{1}_{T},...,F^{K}_{T}$, which can be sorted to derive the $VAR$. We measure the $c$th quantile $Q(F_{T}, c)$ and the average value $Ave(F_{T})$. If $VAR$ is defined as the deviation from the expected value on the target date, we have

$VAR(c) = Ave(F_{t}) - Q(F_{T}, c)$

### Simulation for derivatives

We simply focus on the expected value on the target date discounted into the present:

$F_{t} = e^{-r(T - t)}Ave(F_{T})$

Thus derivatives valuation focuses on the discounted center of the distribution, while $VAR$ focuses on the quantile on the target date.

Monte Carlo simulations have been long used to price derivatives. Pricing derivatives can be done by assuming that the underlying asset grows at the risk-free rate $r$ (assuming no income payment). For instance, pricing an option on a stock with expected return of $20\%$ is done assuming that
1. The stock grows at the risk-free rate of $10\%$.
2. We discount at the same risk-free rate. 

This is called the **risk-neutral approach**.

In contrast, risk measurement deals with actual distributions, sometimes called **physical distributions**. For measuring $VAR$, the risk manager must simulate asset growth using the actual expected return $\mu$ of $20\%$. Therefore, risk management uses physical distributions, whereas pricing methods use risk-neutral distributions.

It should be noted that simulation methods are not applicable to all types of options. These methods assume that the value of the derivative instrument at expiration can be priced solely as a function of the end-of-period price $S_{T}$, and perhaps of its sample path. For example, this is the case with an Asian option, where the payoff is a function of the price averaged over the sample path. Such an option is said to be **path-dependent**.

However, simulation methods are inadequate to price American options, because such options can be exercised early. The optimal exercise decision is complex to model because it should take into account future values of the option. This cannot be done with regular simulation methods, which only consider present and past information. Instead, valing American options requires a **backward recursion**, for example with binomial trees. This method examines whether the option should be exercised, starting from the end and working backward in time until the starting time.

### Accuracy
Then we are going to discuss the effect of **sampling variability**. Unless $K$ is extremely large, the empirical distribution of $S_{T}$ will only be an approximation of the true distribution. There will be some natural variation in statistics measured from Monte Carlo simulations. Since Monte Carlo simulations involve independent draws, one can show that the standard error of statistics is inversely related to the square root of $K$. Thus more simulations will increase precision, but at a slow rate. For example, accuracy is increased by a factor of $10$ going from $K = 10$ to $K = 1,000$, but then requires going from $K = 1,000$ to $K = 100,000$ for the same factor of $10$.

This accuracy issue is worse for risk management than for pricing, because the quantiles are estimated less precisely than the average. For $VAR$ measures, the precision is also a function of the selected confidence level. Higher confidence levels generate fewer observations in the left tail and hence less precise $VAR$ measures. A $99\%$ $VAR$ using $1,000$ replications should be expected to have only $10$ observations in the left tail, which is not a large number. The $VAR$ estimate is derived from the $10$th and $11$th sorted numbers. In contrast, a $95\%$ $VAR$ is measured from the $15$th and $51$st sorted numbers, which is more precise. In addition, the precision of the estimated quantile depends on the shape of the distribution. Relative to a symmetric distribution, a short option position has negative skewness, or a long left tail. The observations in the left tail therefore will be more dispersed, making is more difficult to estimate $VAR$ precisely.

Various methods are available to speed up convergence:
- **Antithetic variable technique:** uses twice the same sequence of random draws from $t$ to $T$. It takes the original sequence and changes the sign of all their values. This creates twice the number of points in the final distribution of $F_{T}$ without running twice the number of simulations.
- **Control variate technique:** is used to price options with trees when a similar option has an analytical solution. Say that $f_{E}$ is a European option with an analytical solution. Going through the tree yields the values of an American option and a European option, $F_{A} and F_{E}$. We then assume that the error in $F_{A}$ is the same as that in $F_{E}$, which is known. The adjusted value is $F_{A} - (F_{E} - f_{E})$
- **Quasi-random sequences:** also called quasi Monte Carlo ($QMC$), creates draws that are not independent but instead are designed to fill the sample space more uniformly. Simulations have shown that $QMC$ methods converge faster than Monte Carlo methods. In other words, for a fixed number of replications $K$, $QMC$ values will be on average closer to the true value.

The advantage of traditional $MC$ is that it also provides a standard error, which is on the order of $1/\sqrt{K}$ because draws are independent. So, we have an idea of how far the estimate might be from the true value, which is useful in deciding on the number of replications. In contrast, $QMC$ methods give no measure of precision.

## <a name="multiple_sources_of_risk">Multiple Sources of Risk</a>
We now turn to more general case of simulations with many sources of financial risk. Define $N$ as the number of risk factors. If the factors $S_{j}$ are independent, the randomization can be performed independently for each variable. For the $GBM$ model, 

$\Delta S_{j,t} = S_{j,t-1}\mu_{j}\Delta t + S_{j,t-1}{\sigma_{j}\epsilon_{j,t}\sqrt{\Delta t}}$

where the standard normal variables $\epsilon$ are independent across time and factor $j = 1,..., N$.

In general, however, risk factors are correlated. The simulation can be adapted by, first, drawing a set of independent variables $eta$, As an example, with two factors only, we write

$\epsilon_{1} = \eta_{1}$

$\epsilon_{2} = \rho\eta_{1} + (1 - \rho^{2})^{1/2}\eta_{2}$

Here, \rho is the correlation coefficient between the variables $\epsilon$. Because the $\eta$'s have unit variance and are uncorrelated, we verify that the variance of $\epsilon_{2}$ is one, as required

$V(\epsilon_{2}) = \rho^{2}V(\eta_{1}) + [(1 - \rho^{2})^{1/2}]^{2}V(\eta_{2}) = \rho^{2} + (1 - \rho^{2}) = 1$

Furthermore, the correlation between $\epsilon_{1}$ and $\epsilon_{2}$ is given by 

$Cov(\epsilon_{1}, \epsilon_{2}) = Cov(\eta_{1}, \rho\eta_{1} + (1 - \rho^{2})^{1/2}\eta_{2}) = \rho Cov(\eta_{1}, \eta_{1}) = \rho$

Defining $\epsilon$ as the vector of values, we verified that the covariance matrix of $\epsilon$ is 

$V(\epsilon) = \begin{bmatrix} \sigma^{2}(\epsilon_{1}) & Cov(\epsilon_{1}, \epsilon_{2}) \\ Cov(\epsilon_{1}, \epsilon_{2}) & \sigma^{2}(\epsilon_{2})\end{bmatrix} = \begin{bmatrix}1 & \rho \\ \rho & 1 \end{bmatrix} = R$

Note that this covariance matrix, which is the expectation of squared deviations from the mean, can also be written as 

$V(\epsilon) = E[(\epsilon - E(\epsilon)) \times (\epsilon - E(\epsilon))'] = E(\epsilon \times \epsilon')$

because the expectation of $\epsilon$ is $0$. To generalize this approach to many more risk factors, however, we need a systematic way to derive the transformation in the $\epsilon_{1}$ and $\epsilon_{2}$ equations.

### The Cholesky factorization
We would like to generate $N$ joint values of $\epsilon$ that display the correlation structure $V(\epsilon) = E(\epsilon\epsilon') = R$. Because the matrix $R$ is a symmetric real matrix, it can be decomposed into its so-called Cholesky factors:

$R = TT'$

where $T$ is a lower triangular matrix with zeros on the upper right corners (above the diagonal). This is known as the **Cholesky factorization**.

As before we first generate a vector of independent $\eta$. Thus, their covariance matrix is $V(\eta) = I$, where $I$ is the identity matrix with zeros everywhere except for the diagonal.

We then construct the transformed variable $\epsilon = T\eta$. The covariance matrix is now $V(\epsilon) = E(\epsilon \epsilon') = E((T\eta)(T\eta')) = E(R\eta\eta'T') = TE(\eta\eta')T' = TV(\eta)T' = TIT' = TT' = R$. This transformation therefore generates $\epsilon$ variables with the desired correlations.

To illustrate, let us go back to our two-variable case. The correlation matrix can be decomposed into its Cholesky factors:

$\begin{bmatrix}1 & \rho \\ \rho & 1 \end{bmatrix} = \begin{bmatrix} a_{11} & 0 \\ a_{21} & a_{22}\end{bmatrix} \begin{bmatrix} a_{11} & a_{21} \\ 0 & a_{22} \end{bmatrix} = \begin{bmatrix}a_{11}^{2} & a_{11}a_{21} \\ a_{21}a_{11} & a_{21}^{2} + a_{22}^2 \end{bmatrix}$

To find the entries $a_{11}$, $a_{21}$, $a_{22}$, we solve each of the three equations

$a_{11}^{2} = 1$

$a_{11}a_{21} = \rho$

$a_{21}^{2} + a_{22}^2 = 1$

This gives $a_{11} = 1$, $a_{21} = \rho$, and $a_{22} = (1 - \rho^{2})^{1/2}$. The Cholesky factorization is then

$\begin{bmatrix}1 & \rho \\ \rho & 1 \end{bmatrix} = \begin{bmatrix} 1 & 0 \\ \rho & (1 - \rho^{2})^{1/2}\end{bmatrix} \begin{bmatrix} 1 & 0 \\ 0 & (1 - \rho^{2})^{1/2}\end{bmatrix}$

Note that this conforms to

$\begin{bmatrix}\epsilon_{1} \\ \epsilon_{2}\end{bmatrix} = \begin{bmatrix} 1 & 0 \\ \rho & (1 - \rho^{2})^{1/2}\end{bmatrix} \begin{bmatrix} \eta_{1} \\ \eta_{2} \end{bmatrix}$

In practice, this decomposition yields a number of useful insights. The decomposition will fail if the number of independent factors implied in the correlation matrix is less than $N$. For instance, if $\rho = 1$, the two assets are perfectly correlated, meaning that we have twice the same factor. This could be, for instance, the case of two currencies fixed to each other. The decomposition gives $a_{11} = 1$, $a_{21} = 1$, $a_{22} = 0$. The new variables are then $\epsilon_{1} = \eta_{1}$ and $\epsilon_{2} = \eta_{1}$. In this case, the second variable $\eta_{2}$ is totally superfluous and simulations can be performed with one variable only.

### The curse of dimensionality
Modern risk management is about measuring the risk of large portfolios, typically exposed to a large number of risk factors. The problem is that the number of computations increases geometrically with the number of factors $N$. The problem is that the number of computations increases geometrically with the number of factors $N$. The covariance matrix, for instance, has dimensions $N(N + 1)/2$. A portfolio with $500$ variables requires a matrix with $125,250$ entries.

In practice, the risk manager should simplify the number of risk factors, discarding those that do not contribute significantly to the risk of the portfolio. Simulations based on the full set of variables would be inordinately time-consuming. The art of simulation is to design parsimonious experiments that represent the breadth of movements in risk factors.

This can be done by an economic analysis of the risk factors and portfolio strategies. Alternatively, the risk manager can perform a statistical decomposition of the covariance matrix. A widely used method for this is the **principal component analysis (PCA)**, which finds linear combinations of the risk factors that have maximal explanatory power. This type of analysis, which is as much an art as it is a science, can be used to reduce the dimensionality of the risk space.